# Wrapping a c++ class in cython
This example follows the class wrapping example on the cython website  
https://cython.readthedocs.io/en/latest/src/userguide/wrapping_CPlusPlus.html

In [2]:
%%writefile Rectangle.h

#ifndef RECTANGLE_H
#define RECTANGLE_H

namespace shapes {
    class Rectangle {
        public:
            int x0, y0, x1, y1;
            Rectangle();
            Rectangle(int x0, int y0, int x1, int y1);
            ~Rectangle();
            int getArea();
            void getSize(int* width, int* height);
            void move(int dx, int dy);
    };
}

#endif

Overwriting Rectangle.h


In [3]:
%%writefile Rectangle.cpp

#include <iostream>
#include "Rectangle.h"

namespace shapes {

    // Default constructor
    Rectangle::Rectangle () {}

    // Overloaded constructor
    Rectangle::Rectangle (int x0, int y0, int x1, int y1) {
        this->x0 = x0;
        this->y0 = y0;
        this->x1 = x1;
        this->y1 = y1;
    }

    // Destructor
    Rectangle::~Rectangle () {}

    // Return the area of the rectangle
    int Rectangle::getArea () {
        return (this->x1 - this->x0) * (this->y1 - this->y0);
    }

    // Get the size of the rectangle.
    // Put the size in the pointer args
    void Rectangle::getSize (int *width, int *height) {
        (*width) = x1 - x0;
        (*height) = y1 - y0;
    }

    // Move the rectangle by dx dy
    void Rectangle::move (int dx, int dy) {
        this->x0 += dx;
        this->y0 += dy;
        this->x1 += dx;
        this->y1 += dy;
    }
}

Writing Rectangle.cpp


In [4]:
%%writefile Rectangle.pxd
cdef extern from "Rectangle.cpp":
    pass

# Declare the class with cdef
cdef extern from "Rectangle.h" namespace "shapes":
    cdef cppclass Rectangle:
        Rectangle() except +
        Rectangle(int, int, int, int) except +
        int x0, y0, x1, y1
        int getArea()
        void getSize(int* width, int* height)
        void move(int, int)

Writing Rectangle.pxd


In [17]:
%%writefile rect.pyx

# distutils: language = c++

from Rectangle cimport Rectangle
def main():
    rec_ptr = new Rectangle(1, 2, 3, 4)  # Instantiate a Rectangle object on the heap
    try:
        rec_area = rec_ptr.getArea()
    finally:
        del rec_ptr  # delete heap allocated object

    cdef Rectangle rec_stack  # Instantiate a Rectangle object on the stack
    
# distutils: language = c++

from Rectangle cimport Rectangle

# Create a Cython extension type which holds a C++ instance
# as an attribute and create a bunch of forwarding methods
# Python extension type.
cdef class PyRectangle:
    cdef Rectangle c_rect  # Hold a C++ instance which we're wrapping

    def __init__(self, int x0, int y0, int x1, int y1):
        self.c_rect = Rectangle(x0, y0, x1, y1)

    def get_area(self):
        return self.c_rect.getArea()

    def get_size(self):
        cdef int width, height
        self.c_rect.getSize(&width, &height)
        return width, height

    def move(self, dx, dy):
        self.c_rect.move(dx, dy)

Overwriting rect.pyx


In [18]:
%%writefile setup.py

from setuptools import setup
from Cython.Build import cythonize
setup(name="rect_test", ext_modules=cythonize("rect.pyx"))

Overwriting setup.py


In [19]:
!python3 setup.py build_ext --inplace

Compiling rect.pyx because it changed.
[1/1] Cythonizing rect.pyx
/home/jake/anaconda3/envs/SEVN/lib/python3.9/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/jake/SEVN/class_example/rect.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'rect' extension
/home/jake/anaconda3/envs/SEVN/bin/x86_64-conda-linux-gnu-cc -Wno-unused-result -Wsign-compare -DNDEBUG -O2 -Wall -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/jake/anaconda3/envs/SEVN/include -I/home/jake/anaconda3/envs/SEVN/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/jake/anaconda3/envs/SEVN/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sec

In [23]:
import rect as test
vars(test)

{'__name__': 'rect',
 '__doc__': None,
 '__package__': '',
 '__loader__': <_frozen_importlib_external.ExtensionFileLoader at 0x7fc0127b6f10>,
 '__spec__': ModuleSpec(name='rect', loader=<_frozen_importlib_external.ExtensionFileLoader object at 0x7fc0127b6f10>, origin='/home/jake/SEVN/class_example/rect.cpython-39-x86_64-linux-gnu.so'),
 '__file__': '/home/jake/SEVN/class_example/rect.cpython-39-x86_64-linux-gnu.so',
 '__builtins__': <module 'builtins' (built-in)>,
 'PyRectangle': rect.PyRectangle,
 'main': <function rect.main>,
 '__test__': {}}